# Import all needed Python Libraries

In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import ipyfilechooser
from openpyxl import load_workbook
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

# Process EXCEL file containing addresses

In [5]:
# Global variable to store addresses
addresses = []

## Choosing Excel file

In [6]:
# File chooser
chooser = ipyfilechooser.FileChooser()
display(chooser)

FileChooser(path='C:\Users\User\PycharmProjects\geoai', filename='', title='', show_hidden=False, select_desc=…

In [7]:
# # This must be reloaded every time 
# wb = load_workbook(chooser.value)
# sheet = wb.active  # Get active sheet
# data = sheet.values  # Read values

In [ ]:
# addresses_list = []
# for index,row in enumerate(data):    
#     if index >0:        
#         # Read only columns with address info and the unique ID
#         result = ",".join([row[0],row[12], row[13], row[17], row[18], row[22],row[23]])
#         addresses_list.append(result)        

In [9]:
# Load the Excel file
df = pd.read_excel(chooser.value, engine="openpyxl", usecols=[0, 12,13,18,22,23])  # Use engine="openpyxl" for .xlsx files

In [12]:
df.set_index('CodiceIdentificativo', inplace=True)

In [15]:
# Display the first few rows
df.sample(10)

,ViaSenzaCivico,Civico,Comune,SiglaProvincia,Provincia
CodiceIdentificativo,,,,,
F10OGF36A1,VIA GUGLIELMO MARCONI,53,BORGO VENETO,PD,PADOVA
B074UB1URA,VIA LIBERAZIONE,5,TORPE',NU,NUORO
F16N8F33UB,LMAR TRIESTE,4,CAMEROTA,SA,SALERNO
F1E0JF3FVZ,VIA XXV APRILE,56,SORBOLO MEZZANI,PR,PARMA
B0BA4B2OG1,PIAZZA PANICO CARDINALE,4,TRICASE,LE,LECCE
F1L23F3T7C,VIA CASSIA AURELIA,76,CHIUSI,SI,SIENA
B0039B018A,VIA ADRIANO OLIVETTI,SNC,MARCIANISE,CE,CASERTA
F17LCF34YL,STRADA PROVINCIALE DEI SANTI,SNC,CASSINO,FR,FROSINONE
B03FIB1J5T,VIA FRATELLI CERVI,6,SAN GIOVANNI VALDARNO,AR,AREZZO


## Processing Excel rows

## Visualize Processed Data and Convert to Pandas Dataframe

In [2]:
addresses_list

NameError: name 'addresses_list' is not defined

In [1]:
# Function to split ID and address
def split_id_address(entry):
    parts = entry.split(",")
    unique_id = parts[0]
    address = ", ".join(parts[1:])  # Join remaining parts into full address
    return unique_id, address


In [ ]:
# Convert data into a DataFrame
df = pd.DataFrame([split_id_address(entry) for entry in addresses_list], columns=["ID", "Address"])

In [ ]:
df

# Geocoding

In [ ]:
# Initialize Nominatim geocoder if faisl change the string of the user_agent
geolocator = Nominatim(user_agent="geo_app1111")

In [ ]:
# Widget to display progress
output_widget = widgets.Output()
display(output_widget)

In [ ]:
# Function to geocode an address with retry logic and live updates
def geocode_address(address, index):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            lat, lon = round(location.latitude, 6), round(location.longitude, 6)  # Round to 6 decimals
        else:
            lat, lon = None, None
    except GeocoderTimedOut:
        time.sleep(1)
        return geocode_address(address, index)  # Retry on timeout
    
    # Update widget display
    with output_widget:
        clear_output(wait=True)
        print(f"Geocoded {index + 1}/{len(df)}: {address} → {lat}, {lon}")
    
    return lat, lon

## Convert to Pandas Dataframe and rejoin the ID for GIS joining

In [ ]:
# Apply geocoding with tqdm progress bar
df["Latitude"], df["Longitude"] = zip(*[geocode_address(addr, idx) for idx, addr in tqdm(enumerate(df["Address"]), total=len(df))])

# Display final result
display(df)

## Export Geocoded Values to Comma Delimited (csv) or/and Excel (xlsx)

In [ ]:
df.to_csv('output.csv')

In [ ]:
df.to_excel('output.xlsx')

# Convert to GIS Dataset (Point)

In [ ]:
# Convert DataFrame to GeoDataFrame
df.dropna(subset=["Latitude", "Longitude"], inplace=True)  # Remove rows with missing coordinates
df["geometry"] = df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)  # Create Point geometries
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # Set CRS to WGS 84

# Display final GeoDataFrame
display(gdf)

In [ ]:
gdf.explore()

## Export to shapefile

In [ ]:
gdf.to_file('output.shp')